In [1]:
# install packages
!pip install pytesseract
!pip install pdf2image

You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
# import packages
import pytesseract
from pdf2image import convert_from_path
import os
import re
from datetime import datetime
import pandas as pd

In [2]:
# Function to extract text from a PDF file
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  

def extract_text_from_pdf(pdf_path):
    # Convert the PDF to a list of PIL Image objects
    images = convert_from_path(pdf_path)
    text_list = []
    # Loop through each page of the PDF and extract text using Tesseract
    for image in images:
        text = pytesseract.image_to_string(image)
        text_list.append(text)
    # Combine text from all pages into a single string
    text = '\n'.join(text_list)
    return text

In [144]:
# Define the regex pattern to match the date
date_pattern = r'\b\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
council_pattern = r'(?i)(\b[A-Za-z]+\s(?:(\n)Council|(\n)Committee|(\n)COMMITTEE)\b)'
council_pattern2 = r"(?<=session).*?(?=Agenda)"
session_pattern = r'(?P<session>[\w-]+\ssession)'
agenda_pattern = r'(?i)Agenda item (\d+)'
agenda_detail_pattern = r"Agenda item \d+(.*?)(draft resolution|draft(?:\n\n)?resolution|draft(?:\n)?resolution)"
footnote_pattern = r'^(\*|\d(?![./])|\d{1}/)\s.*'
split_pattern = r'draft resolution|draft(?:\n\n)?resolution'
title_pattern = r"1\.\s+\w+(\s+\w+)*"

In [145]:
# Function to classify text into column A to N
def classify_text_to_column(text):
    result = []

    # extract year and date (A and J)
    # Search for the date in the text
    match = re.search(date_pattern, text)
    if match:
        date = match.group(0)
        date_obj = datetime.strptime(date, '%d %B %Y')
        new_date = date_obj.strftime('%m/%d/%Y') #column J
        year = new_date[-4:] #column A
    else:
        date, year = 'N/A'
        print("No date found")
    result.append(year)


    # extract council/committe name (B)
    first_twenty_lines = text.split('\n')[:20]
    for line in first_twenty_lines:
        match = re.search(council_pattern, line)
        if match:
            council = match.group(0)
            break
        else:
            council = 'N/A'
            break
            #print("No council found")
    match = re.search(council_pattern2, text, re.DOTALL)
    if match:
        council = match.group(0).strip()
        council = council.replace('\n', ' ') 
    else:
        council = 'N/A'
    if council == ' ' or council == '':
        council = 'N/A'
    result.append(council)


    # extract session (C)
    match = re.search(session_pattern, text)
    if match:
        session = match.group("session")
    else:
        session = 'N/A'
        print("Session information not found.")
    result.append(session)


    # extract Agenda item (D)
    match = re.search(agenda_pattern, text)
    if match:
        agenda_item = match.group(1)
    else:
        agenda_item = 'N/A'
        print("No match found")
    result.append(agenda_item)


    # extract Agenda detail (E) and countries (F)
    match = re.search(agenda_detail_pattern, text, re.DOTALL)
    if match:
        agenda_item_text = match.group(1).strip()
    parts = agenda_item_text.split("\n\n")
    parts = [s.replace('\n', '') for s in parts]
    result.append(parts[0])
    result.append(parts[1][:-1])


    # delete extracted text
    split_text = re.split(split_pattern, text, flags=re.IGNORECASE)
    remain_text = split_text[1]
    #print(remain_text)


    # extract Body title number (G)
    # Split the text into lines
    lines = remain_text.split('\n')
    # Get the first non-empty line
    first_line = next(line for line in lines if line.strip())
    # Check if the first non-empty character is a number using regular expressions
    match = re.match(r'^\d+', first_line)
    if match:
        # If the first non-empty character is a number, extract it
        number = match.group(0)
    else:
        number = 'N/A'
    result.append(number)


    # Body title (H)
    # split the text into paragraphs
    paragraphs = remain_text.split('\n\n')
    # get the text content of the first paragraph
    first_paragraph = paragraphs[1]
    if first_paragraph == "The General Assembly," or first_paragraph[0] == '1':
        idx = 1
        result.append('N/A')
    else:
        first_paragraph = first_paragraph.replace('\n', ' ') 
        idx = 2
        result.append(first_paragraph)


    # Body text (I)
    # extract the text content from the second non-empty paragraph
    body_text = ''
    footnote = ''
    for paragraph in paragraphs[idx:]:
        #for line in paragraph.split('\n'):
            # exclude lines containing specific patterns
        paragraph = paragraph.replace("\n", " ")
        match = re.search(footnote_pattern, paragraph)
        if match:
            if len(match.group(0)) < 50:
                footnote += match.group(0)
        else:
            if not re.search(r'^(\d{2}-\d{5}|[A-Za-z]/\d{2}/[A-Za-z]\.\d|[A-Za-z ]+Please recycle)', paragraph):
                if len(paragraph) > 30 or paragraph == "The General Assembly,":
                    body_text += paragraph.strip() + ' '
    result.append(body_text)

    # date (J)
    result.append(date)
    
    # # file name (K) and country (L) will be obtained outside

    # footnote (M)
    result.append(footnote)

    # scanned (N), cannot know 
    if int(year) > 1990:
        scanned = 'yes'
    else:
        scanned = 'no'
    result.append(scanned)
    
    return result

In [146]:
# Path to the folder containing the PDF files
pdf_folder = './Dataset'

# Path to the folder where the CSV files will be saved
csv_folder = './csv_folder'

# csv file column list
column_list = ['year', 'Council', 'Session', 'Agenda item', 'Agenda detail', 'cosponsored countries', 
               'body title number',	'body title detail', 'body text', 'date', 'footnote', 'scanned', 
               'file', 'filecountry']

# Loop through each folder and PDF file and extract text
for country_folder in os.listdir(pdf_folder):
    print('---------------' + country_folder + '---------------' )
    if not os.path.isdir(os.path.join(pdf_folder, country_folder)):
        continue
    csv_path = os.path.join(csv_folder, f"{country_folder}.csv")
    rows = []
    count = 0
    # Loop through each file 
    for pdf_file in os.listdir(os.path.join(pdf_folder, country_folder)):
        if not pdf_file.endswith('.pdf'):
            continue
        # check if this file is old-version
        year = int(pdf_file.split('_')[0])
        if year < 1994:
            continue

        pdf_path = os.path.join(pdf_folder, country_folder, pdf_file).replace('\\', '/')
        print(pdf_path)
        # try:
        text = extract_text_from_pdf(pdf_path)
        text_columns = classify_text_to_column(text)
        row = text_columns + [pdf_file[:-4], country_folder] #add column K and L
        rows.append(row)
        count += 1
        #print(rows)
        df = pd.DataFrame(rows, columns = column_list)
        df.to_csv(csv_path, index = False) 
        if count == 30:
            break
        # except:
        #     print(f"Error extracting text from {pdf_path}")
        #     continue
    break
    # Write the rows to the CSV file
    df = pd.DataFrame(rows, columns = column_list)
    df.to_csv(csv_path, index = False)  

---------------Afghanistan---------------
./Dataset/Afghanistan/1994_163336.pdf


PermissionError: [Errno 13] Permission denied: './csv_folder\\Afghanistan.csv'

In [147]:
test

'UNITED\nNATIONS A\n\n@) General Assembly oiste,\n—— 3\n\nLIMITED\n\nA/C.3/49/L.20\n15 November 1994\n\nORIGINAL: ENGLISH\n\nForty-ninth session\nTHIRD COMMITTEE\nAgenda item 102\n\nPREPARATION FOR AND ORGANIZATION OF THE UNITED NATIONS\nYEAR FOR TOLERANCE\n\nAfghanistan, Albania, Algeria, Azerbaijan, Benin, Chile,\n\nCosta Rica, Egypt, India, Kyrgyzstan, Morocco, Pakistan,\n\nPeru, Senegal, Togo, Tunisia, Turkey and Ukraine: draft\nresolution\n\nUnited Nations Year for Tolerance\n\nThe General Assembly,\n\nRecalling its resolution 48/126 of 20 December 1993, by which it proclaimed\n1995 the United Nations Year for Tolerance,\n\nRecalling also that the Charter of the United Nations affirms in its\npreamble that to practise tolerance is one of the principles to be applied to\nattain the ends pursued by the United Nations of preventing war and maintaining\npeace,\n\nStressing that one of the purposes of the United Nations as set forth in\nthe Charter is the achievement of international c

In [142]:
test = extract_text_from_pdf('./Dataset/Afghanistan/1994_164949.pdf')

In [148]:
classify_text_to_column(test)

IndexError: list index out of range